In [165]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import contractions

In [166]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [167]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mateusz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Mateusz\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Mateusz\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [168]:
def remove_non_ascii(text):
    # Encode text to ASCII bytes, ignoring errors, then decode back to string
    return text.encode('ascii', 'ignore').decode('ascii')

In [169]:
def replace_special_chars(text):
    # Replace '?' with 'ques_mark', '!' with 'excl_mark', '%' with 'percentage', and '$' with 'dollar'
    text = text.replace('?', ' ques_mark ')
    text = text.replace('!', ' excl_mark ')
    text = text.replace('%', ' percentage ')
    text = text.replace('$', ' dollar ')
    return text

In [170]:
def clean_text(text):
    # 1. Replace special chars
    text = replace_special_chars(text)
    # 2. removing non ascii
    text = remove_non_ascii(text)
    # 3. Lowercasing
    text = text.lower()
    # 4. Replacing US -> USA
    text = text.replace("us", "usa")
    # 5. Removing English possessive forms (’s)
    text = text.replace("'s", "").replace("’s", "")
    # 6. Removing whitespaces
    text = " ".join(text.split())
    # 7. Removing stop words and punctuation
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [word for word in tokens if word not in string.punctuation]
    # 8. Lemmatization
    text = " ".join([lemmatizer.lemmatize(word) for word in tokens])
    # 9. Replacing we'll -> we will
    text = contractions.fix(text)
    return text



In [171]:
df_merged = pd.read_csv("../data/merged_datasetes_balanced_huge.csv")

In [173]:
df_merged['title_cleaned'] = df_merged["title"].apply(clean_text)

In [174]:
pd.set_option('display.max_colwidth', None)  # Show full content of each column
pd.set_option('display.expand_frame_repr', False)  # Disable line wrapping

In [175]:
df_merged.tail(10)

,title,clickbait,set,char_capital,char_small,char_special,blank,numerical,letters,outlier,title_cleaned
39990,6 Celebrities Who Have Embraced Donald Trump as Our President [Video],1,2,9,47,2,10,1,56,0,6 celebrity embraced donald trump president [video]
39991,This District Magistrate Had A Meal Cooked By A Widow To Dispel Villagers' Superstitions,1,1,14,60,1,13,0,74,0,district magistrate meal cooked widow dispel villagers' superstition
39992,Amy Poehler Hiding From Donald Trump Is All Of Us,1,1,10,30,0,9,0,40,0,amy poehler hiding donald trump usa
39993,"The Cast Of ""Grease: Live!"" Competes In A Bubblegum Blowing Competition",1,1,11,46,4,10,0,57,0,"cast ""grease: live excl_mark competes bubblegum blowing competition"
39994,Pigskin A Blanket: NFL Week 11 Picks,1,1,8,19,1,6,2,27,0,pigskin blanket: nfl week 11 pick
39995,This Pregnant Mom Took Her Love Of Starbucks To The Next Level,1,2,12,39,0,11,0,51,0,pregnant mom took love starbucks nextlevel
39996,These Official Harry Potter Anime Characters Will Make You Squeal With Joy,1,1,12,51,0,11,0,63,0,official harry potter anime character make squeal joy
39997,Mapping $27 Billion In Federal Funding Of America's Sanctuary Cities,1,2,9,46,2,9,2,55,0,mapping dollar 27 billion federal funding america sanctuary city
39998,24 Forgotten Items All Late '90s Teen Girls Were Slightly Obsessed With,1,1,10,45,1,11,4,55,0,24 forgotten item late '90s teen girl slightly obsessed
39999,Celebrate World Gin Day with these easy cocktails,1,2,4,38,0,7,0,42,0,celebrate world gin day easy cocktail


In [177]:
df_merged.drop("title",axis=1,inplace=True)
df_merged.rename(columns={"title_cleaned": "title"}, inplace=True)

In [178]:
df_merged = df_merged[df_merged['title'].notna() & (df_merged['title'] != '')]

In [179]:
df_merged.shape

(40000, 10)

In [180]:
df_merged.to_csv("../data/merged_datasetes_balanced_huge_cleaned.csv", index=False)